# Sequence to Sequence (구글 번역의 신경망 모델)
* 순차적인 정보를 입력받는 신경망(RNN)과 출력하는 신경망을 조합한 모델로, 번역이나 챗봇 등 문장을 입력받아 다른 문자을 출력하는 프로그램에서 많이 사용.

<br>

## Seqeuence to Sequence 모델
<img src="https://sunghan-kim.github.io/assets/images/book/3minDL/ch10-05-seq2seq-concept.jpg">

* 입력을 위한 신경망인 인코더와 출력을 위한 신경망인 디코더로 구성된다.
* **go** : 디코더에 입력이 시작됨을 알려줌

* **eos** : 디코더에 출력이 끝났음을 알려줌

* **인코더로 원문을 받으면 디코더로 부터 번역한 결과물을 받는다**

---

<br>
<br>

## 네 글자의 영어 단어를 입력받아 두 글자의 한글 단어로 번역하는 프로그램

<img src="https://sunghan-kim.github.io/assets/images/book/3minDL/ch10-06-seq2seq-translate-model.jpg">

* **<?>** : 심볼
  * **S** : 디코더의 시작
  * **E** : 디코더의 끝
  * **P** : 아무 의미 없는 심볼

---

<br>

## 데이터 만들기
1. 단어를 원-핫 인코딩 형식으로 바꿔야 하므로 char_arr 에 글자들을 나열한다.
2. 한 글자, 한 글자의 인덱스 값을 지정해서 딕셔너리 형태로 num_dic에 저장한다.
3. 글자개수를 dic_len에 저장
4. 학습할 단어들 저장

In [2]:
import tensorflow as tf
import numpy as np

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'],
           ['game', '놀이'], ['girl', '소녀'],
           ['kiss', '키스'], ['love', '사랑']]

---

<br>

## 함수 생성
* **입력 단어와 출력 단어를 매개변수로 받아 인코더의 입력값, 디코더의 입력값과 출력값을 반환하는 함수 생성**
  1. 인코더 셀의 입력값을 위해 입력 단어를 한 글자씩 떼어 배열로 만든다.
  2. 디코더 셀의 입력값을 위해 출력 단어의 글자들을 배열로 만들고, 시작을 나타내는 심볼 'S'를 맨 앞에 붇인다.
  3. 학습을 위해 비교할 디코더 셀의 출력값을 만들고, 출력의 끝을 알려주는 심볼 'E'를 마지막에 붙인다.
  4. 만든 데이터들을 원-핫 인코딩한다.
  
> 손실 함수가 실측값을 원-핫 인코딩한 형태로 자동 변환하여 계산해주기 때문에 디코더 출력값은 인덱스 값을 사용한다.

In [11]:
# 예시
test_data = ['word', '단어']
test_input = []
test_output = []
test_target = []

input = [num_dic[n] for n in test_data[0]]
print('input: ', input)

output = [num_dic[n] for n in ('S' + test_data[1])]
print('output: ', output)

target = [num_dic[n] for n in (test_data[1] + 'E')]
print('target: ', target)

test_input.append(np.eye(dic_len)[input])    
test_output.append(np.eye(dic_len)[output])
test_target.append(target)

print()
print('input one-hot: ', test_input, '\n')
print('output one-hot: ', test_output, '\n')
print('target index data: ', test_target, '\n')

input:  [25, 17, 20, 6]
output:  [0, 29, 30]
target:  [29, 30, 1]

input one-hot:  [array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])] 

output one-hot:  [array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [12]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

---

<br>

## 신경망 모델을 구성할 변수들 정의
* 하이퍼파라미터, 입출력 변수용 수치 정의
* n_class 와 n_input은 입출력에 사용할 글자들의 배열 크기인 dic_len(글자수)이다.

In [14]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len

---

<br>

## 신경망 모델 구성
* **플레이스홀더 정의**
  * **인코더 입력 플레이스홀더, 디코더의 입력 플레이스홀더** : 원-핫 인코딩을 입력으로 사용
  * **디코더 출력 플레이스홀더** : 인덱스 숫자를 사용

In [15]:
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

---

<br>

## RNN 모델을 위한 셀 구성
* **주의할 점** : 디코더를 만들 때 디코더의 초기 상태 값으로 인코더의 최종 상태 값을 넣어줘야 한다. 인코더에서 계산한 상태를 디코더로 전파하기 위함이다.

In [17]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell,
                                            output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                           dtype=tf.float32)
    
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell,
                                            output_keep_prob=0.5)
    
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                           initial_state=enc_states, dtype=tf.float32)

W0801 12:48:35.089006 4658525632 deprecation.py:323] From <ipython-input-17-0aedb3fe390b>:2: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
W0801 12:48:35.093466 4658525632 deprecation.py:323] From <ipython-input-17-0aedb3fe390b>:7: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0801 12:48:35.145265 4658525632 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of pa

---

<br>

## 출력층, 손실 함수, 최적화 함수 구성

In [19]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

W0801 12:52:29.377341 4658525632 deprecation.py:323] From <ipython-input-19-ea2126ae9a2e>:1: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


---

<br>

## 학습
* **feed_dict**
  * 인코더의 입력값, 디코더의 입력값과 출력값

In [20]:
sess  = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                          feed_dict={enc_input: input_batch,
                                    dec_input: output_batch,
                                    targets: target_batch})
    print('Epoch: ', '%04d' %(epoch + 1),
         'cost = ', '{:.6f}'.format(loss))
    
print('최적화 완료!')

Epoch:  0001 cost =  3.770413
Epoch:  0002 cost =  2.841233
Epoch:  0003 cost =  1.632053
Epoch:  0004 cost =  1.204901
Epoch:  0005 cost =  0.915784
Epoch:  0006 cost =  0.429960
Epoch:  0007 cost =  0.416665
Epoch:  0008 cost =  0.234133
Epoch:  0009 cost =  0.275301
Epoch:  0010 cost =  0.157170
Epoch:  0011 cost =  0.094532
Epoch:  0012 cost =  0.078004
Epoch:  0013 cost =  0.039385
Epoch:  0014 cost =  0.051178
Epoch:  0015 cost =  0.064218
Epoch:  0016 cost =  0.047166
Epoch:  0017 cost =  0.010996
Epoch:  0018 cost =  0.017458
Epoch:  0019 cost =  0.022575
Epoch:  0020 cost =  0.103357
Epoch:  0021 cost =  0.003816
Epoch:  0022 cost =  0.010939
Epoch:  0023 cost =  0.007844
Epoch:  0024 cost =  0.009583
Epoch:  0025 cost =  0.002845
Epoch:  0026 cost =  0.001567
Epoch:  0027 cost =  0.004725
Epoch:  0028 cost =  0.003765
Epoch:  0029 cost =  0.006425
Epoch:  0030 cost =  0.008225
Epoch:  0031 cost =  0.001264
Epoch:  0032 cost =  0.006487
Epoch:  0033 cost =  0.004543
Epoch:  00

---

<br>

## 단어 예측 함수 구현
1. 이 모델은 입력값과 출력값 데이터로 (영어 단어, 한글 단어)를 사용하지만 예측 시에는 한글 단어를 알지 못한다. 즉, 단어를 예측하는 것이기 때문에 예측할 단어를 아직 모른다는 의미이다. 따라서 디코더의 입출력을 의미 없는 심볼인 'P'로 채워 넣는다.
  * **ex) ['word', '나무'] 이면 ['word', 'PPPP'] 로 구성이 된다.** 
  
  <br>
  
2. argmax 함수를 통해 가장 확률이 높은 글자의 인덱스들을 추출해 예측값으로 만든다
  * **ex) [[[0, 0.9, 0.1, ...], [0, 0.1, 0.7, ...], ...], ...] => [[[1], [2], ...]]**
  
  <br>
  
3. 예측 결과는 글자의 인덱스를 뜻하기 때무넹, 각 숫자에 해당하는 글자를 가져와 배열로 만든다. 그리고 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
  * **ex) ['l', 'o', 'v', 'e'] 를 입력하게 되면, 디코더의 입력 크기만큼 출력되므로 ['사', '랑', 'E', 'E'] 이 출력된다. 그러므로 'E'부터 그 뒤쪽을 제거하고 문자열로 만들어야만 '사랑' 이라는 단어가 나오게 된다.**

In [21]:
def translate(word):
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)
    
    result = sess.run(prediction,
                     feed_dict={enc_input: input_batch,
                               dec_input: output_batch,
                               targets: target_batch})
    decoded = [cahr_arr[i] for i in result[0]]
    
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated

---

<br>

## 단어 번역

In [22]:
print('\n=== 번역 테스트 ===')

print('word -> ', translate('word'))
print('game -> ', translate('game'))
print('girl -> ', translate('girl'))
print('love -> ', translate('love'))


=== 번역 테스트 ===
word ->  단어
game ->  놀이
girl ->  소녀
love ->  사랑
